In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import threading
from queue import Queue
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

watermark_DIR = "watermark\\WaterM.png"

if not os.path.exists('watermarkdata.csv'):
    pd.DataFrame(columns=['METRIC','PSNR','SSIM','CC','ATTACK','DATASET']).to_csv('watermarkdata.csv', index=False)

In [2]:
def toprint(df,attackNAME):
    columns_to_analyze = df.columns[1:]  # Start from the 2nd column (Column2)
    max_values = df[columns_to_analyze].max()
    min_values = df[columns_to_analyze].min()
    mean_values = df[columns_to_analyze].mean()
    
    # Create a new DataFrame with max, min, and mean values
    output_df = pd.DataFrame({
        'Metric': ['Max', 'Min', 'Mean'],
        'PSNR': [max_values['PSNR'], min_values['PSNR'], mean_values['PSNR']],
        'SSIM': [max_values['SSIM'], min_values['SSIM'], mean_values['SSIM']],
        'CC': [max_values['CC'], min_values['CC'], mean_values['CC']]
    })
    
    # Add a new column 'Dataset' with value 'newdata01'
    output_df['ATTACK'] = attackNAME
    output_df['DATASET'] = "sipi" #___________________________________change dataset name here____________________________
    output_df.to_csv('watermarkdata.csv', mode='a', header=False, index=False)

    return output_df

In [33]:

    
def imperceptibility(images_DIR):


    def calculate_metrics(original_image, water, queue):

        psnr_value = psnr(original_image, water)
        ssim_value = ssim(original_image, water, channel_axis=-1)
        correlation = np.corrcoef(original_image.flatten(), water.flatten())[0, 1]

        queue.put((image_path, psnr_value, ssim_value, correlation))

        return psnr_value, ssim_value, correlation



    def get_image_names(folder_path):
    
        image_names = []
        for filename in os.listdir(folder_path):
            image_names.append(filename)
        return image_names


    
    
    image_names = get_image_names(images_DIR)
    data = {'Image Name': [], 'PSNR': [], 'SSIM': [], 'CC': []}
    queue = Queue() 
    threads = [] 


    for imgname in image_names:
        image_path = images_DIR + imgname
        image = cv2.imread(image_path)
        water = cv2.imread(watermark_DIR)

        # create a thread for each image pair and pass the queue as an argument
        thread = threading.Thread(target=calculate_metrics, args=(image, water, queue))
        threads.append(thread) # add the thread to the list
        thread.start() # start the thread

    for thread in threads:
        thread.join() # wait for all threads to finish

    while not queue.empty(): # get the results from the queue
        image_path, psnr_value, ssim_value, correlation = queue.get()

        data['Image Name'].append(os.path.basename(image_path))
        data['PSNR'].append(psnr_value)
        data['SSIM'].append(ssim_value)
        data['CC'].append(correlation)



    df = pd.DataFrame(data)


    #print(df)
    return df

In [34]:
%%time

images_DIR = "extracted\\"
df=imperceptibility(images_DIR)
toprint(df,'NOattack')


CPU times: total: 375 ms
Wall time: 45 ms


,Metric,PSNR,SSIM,CC,ATTACK
0,Max,41.027525,0.987170,0.999813,NOattack
1,Min,25.068264,0.977933,0.991896,NOattack
2,Mean,39.060178,0.984119,0.999246,NOattack


In [35]:
#df.to_csv('fileWATER.csv')

In [36]:
%%time
images_DIR = "attack\\extracted\\rotation\\"
df=imperceptibility(images_DIR)
toprint(df,'rotation')

CPU times: total: 281 ms
Wall time: 47 ms


,Metric,PSNR,SSIM,CC,ATTACK
0,Max,10.823669,0.758112,0.846111,rotation
1,Min,9.104525,0.608897,0.612454,rotation
2,Mean,10.037417,0.689324,0.738998,rotation


In [37]:
%%time
images_DIR = "attack\\extracted\\downscale\\"
df=imperceptibility(images_DIR)
toprint(df,'downscale')

CPU times: total: 516 ms
Wall time: 47 ms


,Metric,PSNR,SSIM,CC,ATTACK
0,Max,11.286221,0.671011,0.982107,downscale
1,Min,10.991948,0.623446,0.933771,downscale
2,Mean,11.180626,0.650758,0.967158,downscale


In [38]:
%%time
images_DIR = "attack\\extracted\\upscale\\"
df=imperceptibility(images_DIR)
toprint(df,'upscale')

CPU times: total: 297 ms
Wall time: 48 ms


,Metric,PSNR,SSIM,CC,ATTACK
0,Max,15.872154,0.893262,0.997061,upscale
1,Min,15.031373,0.869025,0.988221,upscale
2,Mean,15.754584,0.881666,0.995430,upscale


In [39]:
%%time
images_DIR = "attack\\extracted\\gaussBlurr\\"
df=imperceptibility(images_DIR)
toprint(df,'gaussblurr')

CPU times: total: 516 ms
Wall time: 44.5 ms


,Metric,PSNR,SSIM,CC,ATTACK
0,Max,10.443140,0.609192,0.982911,gaussblurr
1,Min,10.201479,0.582515,0.964657,gaussblurr
2,Mean,10.397685,0.595371,0.975619,gaussblurr


In [40]:
%%time
images_DIR = "attack\\extracted\\saltpepper\\"
df=imperceptibility(images_DIR)
toprint(df,'saltpepper')

CPU times: total: 500 ms
Wall time: 43 ms


,Metric,PSNR,SSIM,CC,ATTACK
0,Max,19.791905,0.883578,0.971336,saltpepper
1,Min,18.186226,0.846209,0.958080,saltpepper
2,Mean,19.106710,0.870205,0.965904,saltpepper


In [41]:
%%time
#30%
images_DIR = "attack\\extracted\\jpeg30\\"
df=imperceptibility(images_DIR)
toprint(df,'jpeg30')


CPU times: total: 375 ms
Wall time: 45 ms


,Metric,PSNR,SSIM,CC,ATTACK
0,Max,10.769013,0.618786,0.777135,jpeg30
1,Min,9.269633,0.471354,0.704611,jpeg30
2,Mean,10.163690,0.561979,0.741825,jpeg30


In [42]:

%%time
#40%
images_DIR = "attack\\extracted\\jpeg40\\"
df=imperceptibility(images_DIR)
toprint(df,'jpeg40')


CPU times: total: 375 ms
Wall time: 49 ms


,Metric,PSNR,SSIM,CC,ATTACK
0,Max,12.849209,0.719683,0.877991,jpeg40
1,Min,11.982337,0.670211,0.818624,jpeg40
2,Mean,12.402728,0.689031,0.846461,jpeg40


In [43]:
%%time
#50%
images_DIR = "attack\\extracted\\jpeg50\\"
df=imperceptibility(images_DIR)
toprint(df,'jpeg50')

CPU times: total: 406 ms
Wall time: 49 ms


,Metric,PSNR,SSIM,CC,ATTACK
0,Max,14.646523,0.777799,0.917236,jpeg50
1,Min,13.525978,0.737587,0.874440,jpeg50
2,Mean,13.990090,0.755407,0.894842,jpeg50


In [44]:

%%time
#60%
images_DIR = "attack\\extracted\\jpeg60\\"
df=imperceptibility(images_DIR)
toprint(df,'jpeg60')


CPU times: total: 406 ms
Wall time: 47 ms


,Metric,PSNR,SSIM,CC,ATTACK
0,Max,16.570882,0.827184,0.945969,jpeg60
1,Min,14.622681,0.762977,0.901942,jpeg60
2,Mean,15.721656,0.804361,0.929360,jpeg60


In [45]:

%%time
#70%
images_DIR = "attack\\extracted\\jpeg70\\"
df=imperceptibility(images_DIR)
toprint(df,'jpeg70')


CPU times: total: 281 ms
Wall time: 47 ms


,Metric,PSNR,SSIM,CC,ATTACK
0,Max,19.181898,0.865441,0.969102,jpeg70
1,Min,16.134127,0.802668,0.932788,jpeg70
2,Mean,17.833328,0.840158,0.956077,jpeg70


In [46]:

%%time
#80%
images_DIR = "attack\\extracted\\jpeg80\\"
df=imperceptibility(images_DIR)
toprint(df,'jpeg80')

CPU times: total: 422 ms
Wall time: 47 ms


,Metric,PSNR,SSIM,CC,ATTACK
0,Max,21.618743,0.898234,0.982698,jpeg80
1,Min,17.583858,0.829974,0.953353,jpeg80
2,Mean,19.963099,0.868373,0.972898,jpeg80


In [47]:
%%time
#95% default
images_DIR = "attack\\extracted\\jpeg\\"
df=imperceptibility(images_DIR)
toprint(df,'jpeg95')

CPU times: total: 500 ms
Wall time: 46 ms


,Metric,PSNR,SSIM,CC,ATTACK
0,Max,28.102185,0.927801,0.996111,jpeg95
1,Min,22.701512,0.900107,0.985901,jpeg95
2,Mean,25.831624,0.915281,0.992982,jpeg95


In [48]:

%%time

images_DIR = "attack\\extracted\\bright05\\"
df=imperceptibility(images_DIR)
toprint(df,'bright05')

CPU times: total: 312 ms
Wall time: 45 ms


,Metric,PSNR,SSIM,CC,ATTACK
0,Max,11.315497,0.661126,0.999454,bright05
1,Min,11.010891,0.648183,0.991394,bright05
2,Mean,11.284157,0.658718,0.998871,bright05


In [49]:
%%time

images_DIR = "attack\\extracted\\bright06\\"
df=imperceptibility(images_DIR)
toprint(df,'bright06')

CPU times: total: 625 ms
Wall time: 46 ms


,Metric,PSNR,SSIM,CC,ATTACK
0,Max,13.255015,0.789651,0.999621,bright06
1,Min,12.758414,0.775123,0.991556,bright06
2,Mean,13.205817,0.786664,0.999048,bright06


In [50]:
%%time

images_DIR = "attack\\extracted\\bright07\\"
df=imperceptibility(images_DIR)
toprint(df,'bright07')

CPU times: total: 391 ms
Wall time: 44 ms


,Metric,PSNR,SSIM,CC,ATTACK
0,Max,15.772902,0.884180,0.999672,bright07
1,Min,14.939028,0.870119,0.991657,bright07
2,Mean,15.674387,0.880955,0.999084,bright07


In [51]:
%%time

images_DIR = "attack\\extracted\\bright08\\"
df=imperceptibility(images_DIR)
toprint(df,'bright08')

CPU times: total: 281 ms
Wall time: 47 ms


,Metric,PSNR,SSIM,CC,ATTACK
0,Max,19.266971,0.945281,0.999702,bright08
1,Min,17.741710,0.932592,0.991670,bright08
2,Mean,19.119844,0.942300,0.999118,bright08


In [52]:
%%time

images_DIR = "attack\\extracted\\bright09\\"
df=imperceptibility(images_DIR)
toprint(df,'bright09')

CPU times: total: 484 ms
Wall time: 48 ms


,Metric,PSNR,SSIM,CC,ATTACK
0,Max,25.202541,0.976629,0.999710,bright09
1,Min,21.464494,0.964808,0.991750,bright09
2,Mean,24.841332,0.973849,0.999137,bright09


In [53]:
%%time

images_DIR = "attack\\extracted\\bright11\\"
df=imperceptibility(images_DIR)
toprint(df,'bright11')

CPU times: total: 406 ms
Wall time: 46 ms


,Metric,PSNR,SSIM,CC,ATTACK
0,Max,29.944520,0.975831,0.998413,bright11
1,Min,25.106523,0.956428,0.992004,bright11
2,Mean,29.272953,0.970324,0.997614,bright11


In [54]:
%%time

images_DIR = "attack\\extracted\\bright12\\"
df=imperceptibility(images_DIR)
toprint(df,'bright12')

CPU times: total: 266 ms
Wall time: 46.2 ms


,Metric,PSNR,SSIM,CC,ATTACK
0,Max,26.877262,0.964636,0.996682,bright12
1,Min,13.947314,0.794619,0.886584,bright12
2,Mean,22.871136,0.915597,0.977620,bright12


In [55]:
%%time

images_DIR = "attack\\extracted\\bright13\\"
df=imperceptibility(images_DIR)
toprint(df,'bright13')

CPU times: total: 297 ms
Wall time: 43 ms


,Metric,PSNR,SSIM,CC,ATTACK
0,Max,24.184520,0.951934,0.993549,bright13
1,Min,8.868193,0.546938,0.624683,bright13
2,Mean,16.757990,0.799753,0.911636,bright13


In [56]:
%%time

images_DIR = "attack\\extracted\\bright14\\"
df=imperceptibility(images_DIR)
toprint(df,'bright14')

CPU times: total: 297 ms
Wall time: 43.3 ms


,Metric,PSNR,SSIM,CC,ATTACK
0,Max,26.587281,0.941689,0.994818,bright14
1,Min,7.769502,0.313805,0.511706,bright14
2,Mean,14.159920,0.678815,0.844152,bright14


In [57]:
%%time

images_DIR = "attack\\extracted\\bright15\\"
df=imperceptibility(images_DIR)
toprint(df,'bright15')

CPU times: total: 297 ms
Wall time: 46 ms


,Metric,PSNR,SSIM,CC,ATTACK
0,Max,24.820982,0.933599,0.991357,bright15
1,Min,6.749012,0.127481,0.474007,bright15
2,Mean,12.386751,0.593277,0.786079,bright15


In [58]:

images_DIR = "attack\\extracted\\crop25UL\\"
df=imperceptibility(images_DIR)
toprint(df,'crop25UL')

,Metric,PSNR,SSIM,CC,ATTACK
0,Max,12.298336,0.745813,0.844542,crop25UL
1,Min,12.070299,0.736605,0.836257,crop25UL
2,Mean,12.254204,0.741710,0.842831,crop25UL


In [59]:
images_DIR = "attack\\extracted\\crop25W\\"
df=imperceptibility(images_DIR)
toprint(df,'crop25W')

,Metric,PSNR,SSIM,CC,ATTACK
0,Max,41.027525,0.987170,0.999813,crop25W
1,Min,25.068285,0.977933,0.991896,crop25W
2,Mean,39.059917,0.984119,0.999246,crop25W


In [60]:
#df.to_csv('fileWATER.csv')